# aprior analysis

### after clustering,it will generate k classes,each class will have a list of user_ids inside that class,next step is to apply basket anaysis on each classification,to get the rules of each class

In [4]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')


In [9]:
order_user_id = orders[['order_id','user_id']]
print(order_user_id)

order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
order_products_id
user_product_id=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')
user_product_id

         order_id  user_id
0         2539329        1
1         2398795        1
2          473747        1
3         2254736        1
4          431534        1
...           ...      ...
3421078   2266710   206209
3421079   1854736   206209
3421080    626363   206209
3421081   2977660   206209
3421082    272231   206209

[3421083 rows x 2 columns]


,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [20]:
user_classification = pd.read_csv('../customer_classification.csv')
user_classification = user_classification[['user_id','label']]
user1 = user_classification[user_classification['label']==0]
user2 = user_classification[user_classification['label']==1]
user3 = user_classification[user_classification['label']==2]
user4 = user_classification[user_classification['label']==3]

In [48]:
user1['user_id']

2            27
11           90
21          152
22          154
25          182
          ...  
40117    206053
40122    206086
40131    206126
40136    206165
40140    206201
Name: user_id, Length: 8048, dtype: int64

In [54]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules
def Apriori(user_data,user_product_id):
    
    user_product_part1 = user_product_id[user_product_id['user_id'].isin(user_data) ]
    print('user_product_part1',user_product_part1)
    user_product_dic={}
    for item in user_product_part1.itertuples():
        if(user_product_dic.get(item[2])):
            user_product_dic[item[2]].append(item[3])
        else:
            user_product_dic[item[2]] = [] 
            user_product_dic[item[2]].append(item[3])
    transaction = []
    for item in user_product_dic:
        transaction.append(user_product_dic[item])
#     print('transaction',transaction)
    te = TransactionEncoder()
    te_ary = te.fit(transaction).transform(transaction)
#     print('te_ary',te_ary)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    t1 = time.time()
#     print('df',df)
    frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
    t2 = time.time()
    print('time t2 - t1',t2-t1)
    return frequent_itemsets
    


In [55]:
frequent_itemsets = Apriori(user1['user_id'],user_product_id)
print(frequent_itemsets)
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

user_product_part1           order_id  user_id  product_id
2630        361493       27       25718
2631       1662354       27       30776
2632       1662354       27        9604
2633       1662354       27        6287
2634       1662354       27       20947
...            ...      ...         ...
33817344   3103285   206201       16315
33817345   3103285   206201       10193
33817346   3103285   206201       38737
33817347   3103285   206201         294
33817348   3103285   206201       39612

[5382247 rows x 3 columns]
time t2 - t1 0.41915178298950195
     support        itemsets
0   0.303554          (4920)
1   0.302063          (5876)
2   0.531933         (13176)
3   0.381958         (16797)
4   0.571819         (21137)
5   0.443961         (21903)
6   0.357232         (22935)
7   0.591327         (24852)
8   0.333126         (24964)
9   0.421098         (26209)
10  0.386556         (27966)
11  0.398981         (39275)
12  0.317470         (40706)
13  0.309891         (42265)
14  0

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(13176),(21137),0.531933,0.571819,0.379100,0.712684,1.246345,0.074931,1.490278
1,(21137),(13176),0.571819,0.531933,0.379100,0.662973,1.246345,0.074931,1.388809
2,(13176),(47209),0.531933,0.462227,0.331759,0.623686,1.349308,0.085886,1.429055
3,(47209),(13176),0.462227,0.531933,0.331759,0.717742,1.349308,0.085886,1.658293
4,(21137),(21903),0.571819,0.443961,0.346173,0.605389,1.363608,0.092307,1.409081
5,(21903),(21137),0.443961,0.571819,0.346173,0.779737,1.363608,0.092307,1.943952
6,(21137),(24852),0.571819,0.591327,0.343812,0.601260,1.016798,0.005680,1.024912
7,(24852),(21137),0.591327,0.571819,0.343812,0.581425,1.016798,0.005680,1.022948
8,(21137),(26209),0.571819,0.421098,0.304175,0.531943,1.263226,0.063383,1.236818
9,(26209),(21137),0.421098,0.571819,0.304175,0.722337,1.263226,0.063383,1.542088
